In [1]:
from pandas import DataFrame, read_csv
from bs4 import BeautifulSoup
import requests
import re
from time import sleep
from random import randrange
from math import sqrt

In [2]:
daf=read_csv("pre_player_2006_2011.csv", encoding = "latin")

In [3]:
player_name=list(daf["name"]) #pl_name
played=list(daf["played"]) #pl_played

#batting order
run_bat=list(daf["run_bat"]) #bat_run
f_50s=list(daf["50s"])  #fi_50s
h_100s=list(daf["100s"]) #hu_100s
batted=list(daf["batted"]) #bat_batted
out=list(daf["out"]) #bat_out

#bowling order
bowled=list(daf["bowled"]) #boll_bowled
wkt_tacken=list(daf["wkt_tacken"]) #boll_wkt
wkt_5=list(daf["5_wkt_taken"]) #boll_5
run_given=list(daf["run_given_bowler"]) #boll_run
econ=list(daf["econ"]) #boll_eco
over=list(daf["over"]) #boll_ove

In [2]:
player_name=[] #pl_name
played=[] #pl_played

#batting order
run_bat=[] #bat_run
f_50s=[]  #fi_50s
h_100s=[] #hu_100s
batted=[] #bat_batted
out=[] #bat_out

#bowling order
bowled=[] #boll_bowled
wkt_tacken=[] #boll_wkt
wkt_5=[] #boll_5
run_given=[] #boll_run
econ=[] #boll_eco
over=[] #boll_ove

In [77]:
play_test=pd.read_csv("player.csv", encoding="latin")

NameError: name 'pd' is not defined

In [115]:
test_name=play_test["name"]
test_played=play_test["played"]
test_bat_run=play_test["run_bat"]
test_50s=play_test["50s"]
test_100s=play_test["100s"]
test_batted=play_test["batted"]
test_out=play_test["out"]
test_bowled=play_test["bowled"]
test_wkt=play_test["wkt_taken"]
test_5wkt=play_test["5_wkt_taken"]
test_boll_run=play_test["run_given_bowler"]
test_econ=play_test["econ"]
test_over=play_test["over"]
#test_country=test_country["country"]

NameError: name 'play_test' is not defined

In [9]:
total=0
match_count=1
for i in range(1, 3):
    newUrl="https://stats.espncricinfo.com/ci/engine/stats/index.html?ballsmax1=300;ballsval1=balls;class=2;filter=advanced;floodlit=1;floodlit=2;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;opposition=1;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;page="+str(i)+";result=1;spanmax2=31+Dec+2011;spanmin2=01+Jan+2011;spanval2=span;team=1;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=team;view=results"
    #print(newUrl)
    response=requests.get(newUrl,timeout=60)
    soup=BeautifulSoup(response.text, 'lxml')
    table = soup.find_all("div",attrs={"class": "engine-dd"})
    res=[]
    for result in table:
        s=result.findAll('a', attrs={'href': re.compile("^/ci/engine/match/")})
        if s:
            href=s[0].get('href')
            res.append("https://stats.espncricinfo.com/"+href)
            
    #bolling and batting calculation
    #batting
    #venue
    table = soup.find_all("div",attrs={"class": "engine-dd"})
    table = soup.find_all('table')[2] 
    table_body=table.find('tbody')
    rows = table_body.find_all('tr')
    winner=[]
    venue_g=[]
    toss=[]
    for row in rows[:]:
        cols=row.find_all('td')
        colm=[]
        for x in cols:
            colm.append(x.text.strip())
        winner.append(colm[0])
        if colm[4]=="won":
            toss.append(colm[0])
        else:
            con=colm[7].split("v ")[1].strip()
            toss.append(con)
        venue_g.append(colm[8].split("(")[0].strip().lower())
    print(len(venue_g))
    print(res[0])
    pointer=0
    for incre in range(len(res)):
        url=res[incre]
        print(url)
        pl_name=[]
        #boll
        pl_played=[]
        boll_eco=[]
        boll_ove=[]
        boll_bowled=[]
        boll_wkt=[]
        boll_5=[]
        boll_run=[]
        #batsman
        bat_run=[]
        fi_50s=[]
        hu_100s=[]
        bat_batted=[]
        bat_out=[]
        not_batted=[]
        team_belong=[]
        
        res_response=requests.get(url, timeout=60)
        res_soup=BeautifulSoup(res_response.text,'lxml')
        
        #team data collected
        team=res_soup.find_all("div", attrs={"class":"cscore_link cscore_link--button"})
        team_name=team[1].find_all("span", attrs={"class":"cscore_name cscore_name--long"})
        teamA=team_name[0].text.strip() #0
        teamB=team_name[1].text.strip() #1
        
        """
        #Venue / Ground
        venue=soup.find("article", attrs={"class":"sub-module game-information pre"})
        venue=venue.find("div", attrs={"class":"stadium-details"})
        venue=venue.find("span")
        venue=venue.text.split(",")[1].strip()
        """
        try:
            print(incre)
            print(venue_g[incre],incre)
            ground_csv=read_csv("ground.csv", encoding = "latin")
            ground=ground_csv[ground_csv["City"]==venue_g[incre]].index.values
            venue=ground_csv["Country"][ground]
            #toss
            tos=toss[incre]
            winner=[incre]
            pointer+=1

            table = res_soup.find_all("article",attrs={"class": "sub-module scorecard"})

            #score of one row
            for score_section in table:
                batting_team=score_section.find("h2").text.strip()
                print("bat",batting_team, teamA, teamB)
                if teamA in batting_team:
                    print("teamA")
                elif teamB in batting_team:
                    print("teamB")
                score_b=score_section.find_all("div", attrs={"class":"wrap batsmen"})
                for result in score_b:
                    #name=result.find("a").text
                    comm=result.find("div", attrs={"class":"cell commentary"})
                    run=result.findAll("div", attrs={"class":"cell runs"})
                    batRun=[]
                    for i in run:
                        if i:
                            batRun.append(i.text)
                    #print(name.text,"--------",comm.text)
                    #how many out
                    bats=result.find("div", attrs={"class":"cell batsmen"})
                    bats=bats.text.split("†")[0].strip().split("(c)")[0].strip().lower()
                    comm=result.find("div", attrs={"class":"cell commentary"})
                    if bats in pl_name:
                        pl_index=pl_name.index(bats)
                        if comm.text!="not out":
                            bat_out[pl_index]+=1
                        if int(batRun[0])>=100:
                            hu_100s[pl_index]+=1
                        elif int(batRun[0])>=50:
                            fi_50s[pl_index]+=1
                        if bat_batted[pl_index]==0:
                            bat_batted[pl_index]=1
                        bat_run[pl_index]+=int(batRun[0])
                    else:
                        try:
                            bat_run.append(int(batRun[0]))
                            pl_name.append(bats)
                            if comm.text!="not out":
                                bat_out.append(1)
                            else:
                                bat_out.append(0)

                            bat_batted.append(1)
                            if int(batRun[0])>=100:
                                hu_100s.append(1)
                                fi_50s.append(0)
                            elif int(batRun[0])>=50:
                                fi_50s.append(1)
                                hu_100s.append(0)
                            else:
                                hu_100s.append(0)
                                fi_50s.append(0)
                            boll_ove.append(0)
                            boll_bowled.append(0)
                            boll_wkt.append(0)
                            boll_5.append(0)
                            boll_run.append(0)
                            boll_eco.append(0)
                            pl_played.append(1)
                            if teamA in batting_team:
                                team_belong.append("teamA")
                            elif teamB in batting_team:
                                team_belong.append("teamB")
                        except:
                                print("error", batRun[0], "type",type(batRun[0]))
                                pass
                #bowling score
                score_bal=score_section.find("div", attrs={"class":"scorecard-section bowling"})
                ball_table=score_bal.find("table")
                body=ball_table.find("tbody")
                rows=body.find_all("tr")
                for row in rows:
                        cols=row.find_all("td")
                        colm=[]
                        for x in cols:
                            colm.append(x.text.strip())
                        colmN=colm[0].split("†")[0].strip().split("(c)")[0].strip().lower()
                        if colmN in pl_name:

                            pl_index=pl_name.index(colmN)
                            boll_ove[pl_index]+=float(colm[2])  #2
                            boll_run[pl_index]+=int(colm[4]) #4
                            boll_wkt[pl_index]+=int(colm[5]) #5
                            if boll_eco[pl_index]:
                                boll_eco[pl_index]=float((boll_eco[pl_index]+float(colm[6]))/2) #6
                            else:
                                boll_eco[pl_index]=float(colm[6])
                            if  int(colm[5])>=5:
                                boll_5[pl_index]+=1
                            if boll_bowled[pl_index]==0:
                                boll_bowled[pl_index]=1

                        else:
                            colmN=colm[0].split("†")[0].strip().split("(c)")[0].strip().lower()
                            pl_name.append(colmN) #0
                            boll_ove.append(float(colm[2]))  #2
                            boll_run.append(int(colm[4])) #4
                            boll_wkt.append(int(colm[5])) #5
                            boll_eco.append(float(colm[6])) #6
                            hu_100s.append(0)
                            fi_50s.append(0)
                            bat_batted.append(0)
                            bat_run.append(0)
                            bat_out.append(0)
                            if  int(colm[5])>=5:
                                boll_5.append(1)
                            else:
                                boll_5.append(0)
                            boll_bowled.append(1)
                            pl_played.append(1)
                            if teamA in batting_team:
                                team_belong.append("teamB")
                            elif teamB in batting_team:
                                team_belong.append("teamA")

                # not batted player
                not_bat=score_section.find("div", attrs={"class":"wrap dnb"})
                cell=not_bat.find("div", attrs={"class":"cell"})
                row=cell.find_all("a")
                if row:
                    for rows in row:
                            play_not=rows.text.replace(',',"")
                            play_not=play_not.strip()
                            play_not=play_not.split("†")[0].strip().split("(c)")[0].strip().lower()
                            if play_not not in pl_name:
                                    pl_name.append(play_not)
                                    boll_ove.append(0)  #2
                                    boll_run.append(0) #4
                                    boll_wkt.append(0) #5
                                    boll_eco.append(0) #6
                                    boll_5.append(0)
                                    boll_bowled.append(0)
                                    hu_100s.append(0)
                                    fi_50s.append(0)
                                    bat_batted.append(0)
                                    bat_run.append(0)
                                    bat_out.append(0)
                                    pl_played.append(1)
                                    if teamA in batting_team:
                                        team_belong.append("teamA")
                                    elif teamB in batting_team:
                                        team_belong.append("+eamB")    
            """
            batt=bat(pl_name, pl_played, bat_batted, hu_100s, fi_50s, bat_run, bat_out, 2012)                            
            batt=batt.model_bat()
            """
            dft=DataFrame()
            dft["name"]=pl_name #0
            dft["boll_bowled"]=boll_bowled
            dft["over"]=boll_ove  #2
            dft["boll_run"]=boll_run #4
            dft["wkt"]=boll_wkt #5
            dft["econ"]=boll_eco #6
            dft["5_wkt"]=boll_5
            dft["bat_run"]=bat_run
            dft["bat_out"]=bat_out
            dft["bat_batted"]=bat_batted
            dft["pl_paly"]=pl_played
            dft["fifties"]=fi_50s
            dft["centuries"]=hu_100s
            #dft["team_belong"]=team_belong
            #dft["batsman_score"]=batt
            #print(len(team_belong), len(pl_name))
            #print(dft.head(22))
            dft.to_csv("C:/Users/HP/winnning/matches/"+str(match_count)+".csv", mode="a",header=True)
            match_count+=1


            #final data updation before moving to next match
            
            for pl in pl_name:
                player_id=pl_name.index(pl)
                if pl in player_name:
                    ind=player_name.index(pl)
                    #name[index] #pl_name
                    played[ind]+=pl_played[player_id]

                    #batting order
                    run_bat[ind]+=bat_run[player_id]
                    f_50s[ind]+=fi_50s[player_id]
                    h_100s[ind]+=hu_100s[player_id]
                    batted[ind]+=bat_batted[player_id]
                    out[ind]+=bat_out[player_id]

                    #bowling order
                    bowled[ind]+=boll_bowled[player_id]
                    wkt_tacken[ind]+=boll_wkt[player_id]
                    wkt_5[ind]+=boll_5[player_id]
                    run_given[ind]+=boll_run[player_id]
                    econ[ind]=(econ[ind]+boll_eco[player_id])/2
                    over[ind]+=boll_ove[player_id]
                else:
                    player_name.append(pl)
                    played.append(pl_played[player_id]) #pl_played

                    #batting order
                    run_bat.append(bat_run[player_id])
                    f_50s.append(fi_50s[player_id])
                    h_100s.append(hu_100s[player_id])
                    batted.append(bat_batted[player_id])
                    out.append(bat_out[player_id])

                    #bowling order
                    bowled.append(boll_bowled[player_id])
                    wkt_tacken.append(boll_wkt[player_id])
                    wkt_5.append(boll_5[player_id])
                    run_given.append(boll_run[player_id])
                    econ.append(boll_eco[player_id])
                    over.append(boll_ove[player_id])
        except:
            print(incre)
            pointer+=1
            print("error: ground")
            print(len(venue_g))
    #total+=len(res)
    ran=randrange(0,4,2)
    sleep(ran)


50
https://stats.espncricinfo.com//ci/engine/match/463150.html
https://stats.espncricinfo.com//ci/engine/match/463150.html
0
durban 0
bat South Africa Innings South Africa India
teamA
bat India Innings South Africa India
teamB
https://stats.espncricinfo.com//ci/engine/match/463151.html
1
johannesburg 1
bat India Innings India South Africa
teamA
bat South Africa Innings India South Africa
teamB
https://stats.espncricinfo.com//ci/engine/match/446962.html
2
melbourne 2
bat England Innings England Australia
teamA
bat Australia Innings England Australia
teamB
https://stats.espncricinfo.com//ci/engine/match/463152.html
3
cape town 3
bat South Africa Innings South Africa India
teamA
bat India Innings South Africa India
teamB
https://stats.espncricinfo.com//ci/engine/match/446963.html
4
hobart 4
bat Australia Innings Australia England
teamA
bat England Innings Australia England
teamB
https://stats.espncricinfo.com//ci/engine/match/463153.html
5
port elizabeth 5
bat South Africa Innings South A

48
port of spain 48
bat West Indies Innings West Indies India
teamA
bat India Innings West Indies India
teamB
https://stats.espncricinfo.com//ci/engine/match/489222.html
49
port of spain 49
bat West Indies Innings West Indies India
teamA
bat India Innings West Indies India
teamB
40
https://stats.espncricinfo.com//ci/engine/match/489223.html
https://stats.espncricinfo.com//ci/engine/match/489223.html
0
north sound 0
bat West Indies Innings West Indies India
teamA
bat India Innings West Indies India
teamB
https://stats.espncricinfo.com//ci/engine/match/489224.html
1
north sound 1
bat West Indies Innings West Indies India
teamA
bat India Innings West Indies India
teamB
https://stats.espncricinfo.com//ci/engine/match/489225.html
2
kingston 2
bat India Innings India West Indies
teamA
bat West Indies Innings India West Indies
teamB
https://stats.espncricinfo.com//ci/engine/match/474467.html
3
the oval 3
bat England Innings England Sri Lanka
teamA
bat Sri Lanka Innings England Sri Lanka
teamB

In [4]:
dff=DataFrame()
dff["name"]=player_name
dff["played"]=played
dff["run_bat"]=run_bat
dff["50s"]=f_50s
dff["100s"]=h_100s
dff["batted"]=batted
dff["out"]=out
dff["bowled"]=bowled
dff["wkt_tacken"]=wkt_tacken
dff["5_wkt_taken"]=wkt_5
dff["run_given_bowler"]=run_given
dff["econ"]=econ
dff["over"]=over

In [7]:
dff.to_csv("pre-2008-2011.csv", mode="a", header=True)

In [6]:
dff

,name,played,run_bat,50s,100s,batted,out,bowled,wkt_tacken,5_wkt_taken,run_given_bowler,econ,over
0,WU Tharanga,93,3101,15,9,93,88,0,0,0,0,0.000000e+00,0.0
1,DA Gunawardene,1,3,0,0,1,1,0,0,0,0,0.000000e+00,0.0
2,KC Sangakkara,135,5122,39,7,133,126,0,0,0,0,0.000000e+00,0.0
3,DPMD Jayawardene,135,4370,31,7,134,121,1,0,0,15,1.410593e-36,2.0
4,MS Atapattu,25,697,4,0,24,21,0,0,0,0,0.000000e+00,0.0
5,TM Dilshan,129,3697,14,7,121,108,82,33,0,1740,4.631514e+00,358.6
6,MF Maharoof,51,678,2,0,40,32,51,67,2,1897,6.189166e+00,381.8
7,RP Arnold,31,615,5,0,29,18,6,3,0,85,6.872292e-01,16.1
8,WPUJC Vaas,55,320,0,0,42,28,55,66,1,1990,4.153355e+00,471.3
9,KD Mills,62,614,2,0,43,33,62,110,1,2545,5.195383e+00,528.8


In [5]:
dff.to_csv("pre-2008-11.csv", mode="a",header=True)

In [8]:
dff

,name,played,run_bat,50s,100s,batted,out,bowled,wkt_tacken,5_wkt_taken,run_given_bowler,econ,over
0,WU Tharanga,93,3101,15,9,93,88,0,0,0,0,0.000000e+00,0.0
1,DA Gunawardene,1,3,0,0,1,1,0,0,0,0,0.000000e+00,0.0
2,KC Sangakkara,135,5122,39,7,133,126,0,0,0,0,0.000000e+00,0.0
3,DPMD Jayawardene,135,4370,31,7,134,121,1,0,0,15,1.410593e-36,2.0
4,MS Atapattu,25,697,4,0,24,21,0,0,0,0,0.000000e+00,0.0
5,TM Dilshan,129,3697,14,7,121,108,82,33,0,1740,4.631514e+00,358.6
6,MF Maharoof,51,678,2,0,40,32,51,67,2,1897,6.189166e+00,381.8
7,RP Arnold,31,615,5,0,29,18,6,3,0,85,6.872292e-01,16.1
8,WPUJC Vaas,55,320,0,0,42,28,55,66,1,1990,4.153355e+00,471.3
9,KD Mills,62,614,2,0,43,33,62,110,1,2545,5.195383e+00,528.8


In [57]:
r=read_csv("2008.csv", encoding = "latin")

In [59]:
x=r[r["name"]=="GC Smith (c)"].index.values

In [61]:
x[0]

11

In [27]:
from pandas import read_csv, DataFrame
from math import sqrt
class bat:
    def __init__(self, pl_name, played, batted, hu_100s, fi_50s, run, out, year):
        self.pl_name=pl_name
        self.played=played
        self.batted=batted
        self.hu_100s=hu_100s
        self.fi_50s=fi_50s
        self.run=run
        self.out=out
        self.year=year

        self.career=[]
        self.recent=[]
        self.batsman_score=[]
    def model_bat(self):
        year1=read_csv(str(self.year-4)+".csv", encoding = "latin")
        year2=read_csv(str(self.year-3)+".csv", encoding = "latin")
        year3=read_csv(str(self.year-2)+".csv", encoding = "latin")
        year4=read_csv(str(self.year-1)+".csv", encoding = "latin")
        for pl in self.pl_name:
            index=self.pl_name.index(pl)
            u=self.batted[index]/self.played[index]
            u=sqrt(u)
            v=(20*self.hu_100s[index])+(5*self.fi_50s[index])
            bat_avg=""
            if self.out[index]!=0:
                bat_avg=self.run[index]/self.out[index]
            else:
                bat_avg=self.run[index]
            w=0.3*v+0.7*bat_avg
            
            #career_score
            self.career.append(w*u)
            
            #last four match
            #recent_score calculation
            total=0
            val=year1[year1["name"]==pl].index.values
            if val:
                total+=year1["run_bat"][val[0]]
            val=year2[year2["name"]==pl].index.values
            if val:
                total+=year2["run_bat"][val[0]]
            val=year3[year3["name"]==pl].index.values
            if val:
                total+=year3["run_bat"][val[0]]
            val=year4[year4["name"]==pl].index.values
            if val:
                total+=year4["run_bat"][val[0]]
            self.recent.append(total/4)
            
        #batsman_score calculate    
        for i in range(len(self.pl_name)):
            #index=self.pl_name.index(pl)
            self.career[i]=self.career[i]/max(self.career)
            self.recent[i]=self.recent[i]/max(self.recent)
            self.batsman_score.append(0.35*self.career[i]+0.65*self.recent[i])
        return self.batsman_score


In [73]:
#new="https://stats.espncricinfo.com/ci/engine/stats/index.html?ballsmax1=300;ballsval1=balls;class=2;filter=advanced;floodlit=1;floodlit=2;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;opposition=1;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;page=%22+str%28i%29+%22;result=1;spanmax2=31+Dec+2011;spanmin2=01+Jan+2006;spanval2=span;team=1;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=team;view=results"
new="https://stats.espncricinfo.com/ci/engine/stats/index.html?ballsmax1=300;ballsval1=balls;class=2;filter=advanced;floodlit=1;floodlit=2;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;opposition=1;opposition=2;opposition=25;opposition=3;opposition=4;opposition=5;opposition=6;opposition=7;opposition=8;orderby=start;page="+str(i)+";result=1;spanmax2=31+Dec+2011;spanmin2=01+Jan+2006;spanval2=span;team=1;team=2;team=25;team=3;team=4;team=5;team=6;team=7;team=8;template=results;type=team;view=results"
response=requests.get(new, timeout=10)
soup=BeautifulSoup(response.text,'lxml')

In [74]:
table = soup.find_all("div",attrs={"class": "engine-dd"})
table = soup.find_all('table')[2] 
table_body=table.find('tbody')
rows = table_body.find_all('tr')
winner=[]
venue=[]
toss=[]
i=0
for row in rows[:]:
    cols=row.find_all('td')
    colm=[]
    for x in cols:
        colm.append(x.text.strip())
    print(colm,i)
    i+=1
    winner.append(colm[0])
    if colm[4]=="won":
        toss.append(colm[0])
    else:
        con=colm[7].split("v ")[1]
        toss.append(con)
    venue.append(colm[8].split("(")[0].strip())


['New Zealand', 'won', '5 wickets', '12', 'won', '2nd', '', 'v Sri Lanka', 'Christchurch', '3 Jan 2006', ''] 0
['New Zealand', 'won', '21 runs', '', 'won', '1st', '', 'v Sri Lanka', 'Wellington', '6 Jan 2006', ''] 1
['Sri Lanka', 'won', '20 runs', '', 'won', '1st', '', 'v New Zealand', 'Napier', '8 Jan 2006', ''] 2
['Australia', 'won', '116 runs', '', 'lost', '1st', '', 'v Sri Lanka', 'Melbourne (Docklands)', '13 Jan 2006', ''] 3
['South Africa', 'won', '5 wickets', '7', 'lost', '2nd', '', 'v Australia', 'Brisbane', '15 Jan 2006', ''] 4
['Sri Lanka', 'won', '94 runs', '', 'lost', '1st', '', 'v South Africa', 'Brisbane', '17 Jan 2006', ''] 5
['Australia', 'won', '59 runs', '', 'won', '1st', '', 'v South Africa', 'Melbourne (Docklands)', '20 Jan 2006', ''] 6
['Sri Lanka', 'won', '51 runs', '', 'won', '1st', '', 'v Australia', 'Sydney', '22 Jan 2006', ''] 7
['South Africa', 'won', '9 runs', '', 'won', '1st', '', 'v Sri Lanka', 'Adelaide', '24 Jan 2006', ''] 8
['Australia', 'won', '5 wicke

In [75]:
df=DataFrame({"winner":winner, "toss":toss,"venue":venue})
df

,toss,venue,winner
0,New Zealand,Christchurch,New Zealand
1,New Zealand,Wellington,New Zealand
2,Sri Lanka,Napier,Sri Lanka
3,Sri Lanka,Melbourne,Australia
4,Australia,Brisbane,South Africa
5,South Africa,Brisbane,Sri Lanka
6,Australia,Melbourne,Australia
7,Sri Lanka,Sydney,Sri Lanka
8,South Africa,Adelaide,South Africa
9,Sri Lanka,Adelaide,Australia


In [15]:
"Colombo (SSC)".split("(")

['Colombo ', 'SSC)']

In [63]:
venue[3]

'Melbourne'

In [83]:
Unew="https://www.espncricinfo.com/series/14685/scorecard/238176/new-zealand-vs-west-indies-2nd-odi-west-indies-tour-of-new-zealand-2005-06"
#Unew="https://www.espncricinfo.com/series/8531/scorecard/249229/australia-vs-new-zealand-5th-match-commonwealth-bank-series-2006-07"
response=requests.get(Unew, timeout=10)
soup=BeautifulSoup(response.text,'lxml')
team=soup.find_all("div", attrs={"class":"cscore_link cscore_link--button"})
team_name=team[1].find_all("span", attrs={"class":"cscore_name cscore_name--long"})
teamA=team_name[0].text.strip() #0
teamB=team_name[1].text.strip() #1

In [85]:
teamA

'West Indies'

In [112]:
randrange(12,20,2)

14